The first part to be done in this project is the conversion of provided R Code into Python format. The code was provided by my supervisor and it had to be analysed and rewritten using Python modules, pandas, and functions. The first section deals with the modules added to replicate the R packages

In [1]:
#library(R.matlab) is run as [from scipy.io import loadmat]
#library(hyperSpec) done through hyperspy and spectral modules
#library(ChemometricsWithR) is obtained through scipy modules and scikit-learn. 
#library(baseline) is part of scipy module
#library(ggplot2) is part of plotnine module
#library(pls) is part of sklearn module

import pandas as pd
import numpy as np
import peakutils
from scipy.io import loadmat
from scipy import signal, misc #Replaces signal library
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from BaselineRemoval import BaselineRemoval
from peakutils import baseline 
import math
from sklearn.preprocessing import normalize

After loading the main modules, we follow with the loading of the provided Mat format data. This requires converting the files into lists and setting named variables of the files to the array

In [6]:
#Loading wavenumber data
wavenumber = loadmat("wavenumber.mat")
wavenumber = wavenumber['wavenumber']

#LNCaP
data = loadmat("LNCaP_0Gy.mat") #This loads the matlab file into the workspace as a 'list' object
LNCaP_0Gy = data['data'] #Extracting the LNCaP spectral data matrix from R Code

data = loadmat("LNCaP_IF.mat") #This loads the matlab file into the workspace as a 'list' object
LNCaP_IF = data['data'] 

[[ 750  752  754 ... 3996 3998 4000]]


Savitzky–Golay is then applied to the data, however difficulty is experienced trying to covert the idea of setting the vector to a null object, likely one of the functions is not correct for the type of vector data used. This process was done over the two data sets

In [3]:
#SG Filtering
numrow = LNCaP_0Gy.shape[0] # Shape functions as nrow and ncol but specificed to the array elements
numcol = LNCaP_0Gy.shape[1]
vec1 = {} # None is THE Python equivalent of R's Null object
vec1[tuple(np.linspace(1,numrow))] = "LNCaP 0Gy" #linspace is seq for use in Python


filtLNCaP_0Gy = np.asmatrix(LNCaP_0Gy) # Should work in creating similar matrix

from scipy.signal import savgol_filter

for i in range(1,numrow): #Removed {} and got sg filter from python, ordering is n,p,m versus p,n,m

  filtLNCaP_0Gy[i,] == savgol_filter(LNCaP_0Gy[i,], 13, 5, 0)


Several baseline methods were tested, however none have been able to work so far. There may be an issue based in trying to copy the R code to closely, creating an array that python Baseline functions / methods cannot process. Looking back on the array data to find an issue may be able to get the baseline correction functions to work, however this understanding of the array form will require research as to why the original methods were used

In [ ]:
#Baseline Correction Method 1
#filtLNCaP_0Gy = filtLNCaP_0Gy[0:10, 0:10]
#scaler = MinMaxScaler(feature_range=(-1, 1))
#filtLNCaP_0Gy = scaler.fit_transform(filtLNCaP_0Gy.reshape(-1, 1))
#LNCaP_0Gy = BaselineRemoval(filtLNCaP_0Gy[0], 1626)
#LNCaP_0Gy_corrected = LNCaP_0Gy.IModPoly()
#LNCaP_0Gy_corrected.type

#for array in np.asarray(filtLNCaP_0Gy):
  #LNCaP_0Gy = BaselineRemoval(array, 1626)
  #LNCaP_0Gy_corrected = LNCaP_0Gy.IModPoly()
  #LNCaP_0Gy_corrected.type
    # Takes unknown amount of time to execute (8 hours running did not reach end of execution of code)

#Baseline Correction Method 2
#LNCaP_0Gy = peakutils.baseline(filtLNCaP_0Gy) 
  #Creates error 'shapes (4,2738184) and (1684,1626) not aligned: 2738184 (dim 1) != 1684 (dim 0)'


#Baseline Correction Method 3
#from scipy import sparse
#from scipy.sparse.linalg import spsolve  
  
#def baseline_als(y, lam, p, niter=10):
  #L = len(y)
  #D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
  #w = np.ones(L)
  #for i in range(niter):
    #W = sparse.spdiags(w, 0, L, L)
    #Z = W + lam * D.dot(D.transpose())
    #z = spsolve(Z, w*y)
    #w = p * (y > z) + (1-p) * (y < z)
  #return z

#LNCaP_0Gy = baseline_als(filtLNCaP_0Gy, 2, 5)
  # Creates error 'matrix - rhs dimension mismatch ((1684, 1684) - 1)'
  

In [ ]:
#SG Filtering for LNCAP_IF
numrow1 = LNCaP_IF.shape[0]
numcol1 = LNCaP_IF.shape[1]
vec2= {}
vec2[tuple(np.linspace(1,numrow1))] = "LNCaP 2Gy"

filtLNCaP_IF = np.asmatrix(LNCaP_IF)

for i in range(1,numrow1):
    
  filtLNCaP_IF[i,] == savgol_filter(LNCaP_IF[i,],13, 5, 0)

#Baseline Correction
##LNCaP_IF = baseline(filtLNCaP_IF, 2, hwi=210,  it=5,  int=1050, method='fillPeaks')
##LNCaP_IF = LNCaP_IF@corrected

Currently, Baseline correction has not been able to work with two methods attempted. The issue arrives as there is no Python equivalent to the baseline package in R, which uses 4S Peak Filling – baseline estimation by iterative mean suppression, a paper of which can be found at [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4487348/] however the algorithm is not provided besides the R package that is the basis of the baseline function used in the R code.

The savgol filter hasnt been confirmed if it functions as the R code does, but the code that does work appears to run identical (not working code is given ##)

In [4]:
ID = np.concatenate((vec1, vec2), axis=None).astype(str) #ID = as.character(c(vec1,vec2)), axis must be set to None to create singe dimension vector, then it is set as string type

#NORMALIZE spectra (vector normalization)
##LNCaP_0Gy = sweep(LNCaP.0Gy, MARGIN=1, apply(LNCaP_0Gy,1,function(x) sqrt(sum(x^2))) , FUN="/") #Vector normalisation function(x)sqrt(sum(x^2))
##LNCaP_IF = sweep(LNCaP.IF, MARGIN=1, apply(LNCaP_IF,1,function(x) sqrt(sum(x^2))) , FUN="/") #Vector normalisation function(x)sqrt(sum(x^2))

#Normalize Spectra Method 1
##df2 = np.sqrt(np.sum(np.square(LNCaP_0Gy), axis=0))
##LNCaP_0Gy = pd.Series(range(LNCaP_0Gy.shape[0])).apply(lambda row_count: np.divide(LNCaP_0Gy[row_count,:],df2[row_count,]))

dataall = np.concatenate([LNCaP_0Gy,LNCaP_IF])

Concatenating vec1 and vec2 into a string vector was succesful, and similarly done with LNCaP_0Gy and LNCap_IF for datall. The Spectra Normalization was running but when tested, the values did not correpond to the same point as on the R code, even when compensating and not running the baseline code. 

Trying to fix it led to issues with the array being out of bounds ("index 1626 is out of bounds for axis 0 with size 1626")
Will attempt to redo the code to figure out how to keep it in bounds

The main issue was as with the baseline package, there was no equivalent form of the sweep function in Python. The code used was based on similar attempts, posted online, of replicating the function in Python

In [5]:
#PCA
##PCA_Data = <- prcomp(dataall ,center = TRUE,scale = FALSE)

##fviz_pca_ind(data.pca,label="none",title="",habillage = ID, addEllipses = TRUE, ellipse.level = 0.95)+
  #theme_minimal()+
  #xlab("PC1 (52.9%)")+
  #ylab("PC2 (19.3%)")+
  #scale_color_brewer(palette="Set1")


IndentationError: unexpected indent (<ipython-input-5-0d2ee95f5730>, line 5)

PCA analysis was also unable to determine a similar method, and it relied on the data from previous sections, so baseline correction and Normalizing was prioritised

In [ ]:
#PNT1A

###################First two data sets
data = loadmat("PNT1A_0Gy.mat") #This loads the matlab file into the workspace as a 'list' object
PNT1A_0Gy = data['data']  #Extracting the PNT1A PNT1A.0Gytral data matrix

data = loadmat("PNT1A_OF.mat") #This loads the matlab file into the workspace as a 'list' object
PNT1A_OF = data['data'] 

#SG Filtering 
numrow = PNT1A_0Gy.shape[0]
numcol = PNT1A_0Gy.shape[1]

    filtPNT1A_0Gy = np.asmatrix(PNT1A_0Gy)

for i range(1,numrow):
    
  filtPNT1A_0Gy[i,] == savgol_filter(PNT1A_0Gy[i,],13, 5, 0)

#Baseline Correction / basline.fill peaks method 4s Peaks method
##PNT1A.0Gy <- baseline(filtPNT1A.0Gy, lambda=2, hwi=210,  it=5,  int=1050,method='fillPeaks')
##PNT1A.0Gy <- PNT1A.0Gy@corrected

#SG Filtering 
numrow1 = PNT1A_OF.shape[0]
numcol1 = PNT1A_OF.shape[1]

filtPNT1A_OF = np.asmatrix(PNT1A_OF)

for i range(1,numrow1):
    
  filtPNT1A_OF[i,] == savgol_filter(PNT1A_OF[i,],13, 5, 0)

#Baseline Correction
##PNT1A.OF <- baseline(filtPNT1A.OF, lambda=2, hwi=210,  it=5,  int=1050,method='fillPeaks')
##PNT1A.OF <- PNT1A.OF@corrected

#NORMALIZE spectra (vector normalization)
##PNT1A.0Gy=sweep(PNT1A.0Gy, MARGIN=1, apply(PNT1A.0Gy,1,function(x) sqrt(sum(x^2))) , FUN="/") #Vector normalisation function(x)sqrt(sum(x^2))
##PNT1A.OF=sweep(PNT1A.OF, MARGIN=1, apply(PNT1A.OF,1,function(x) sqrt(sum(x^2))) , FUN="/") #Vector normalisation function(x)sqrt(sum(x^2))

#######################Second two data sets
data = loadmat("PNT1A_0Gy_ICCM.mat") #This loads the matlab file into the workspace as a 'list' object
PNT1A_0Gy_ICCM = data['data']  #Extracting the spectral data matrix

data = loadmat("PNT1A_OF_ICCM.mat") #This loads the matlab file into the workspace as a 'list' object
PNT1A_OF_ICCM = data['data'] 

#SG Filtering
numrow = PNT1A_OF_ICCM.shape[0]
numcol = PNT1A_OF_ICCM.shape[1]

filtPNT1A_0Gy_ICCM = np.asmatrix(PNT1A_0Gy_ICCM)


for i range(1,numrow):
    
  filtPNT1A_0Gy_ICCM[i,] == savgol_filter(PNT1A_0Gy_ICCM[i,],13, 5, 0)

#Baseline Correction
##PNT1A.0Gy.ICCM <- baseline(filtPNT1A.0Gy.ICCM, lambda=2, hwi=210,  it=5,  int=1050,method='fillPeaks')
##PNT1A.0Gy.ICCM <- PNT1A.0Gy.ICCM@corrected

#SG Filtering
numrow1 = PNT1A_OF_ICCM.shape[0]
numcol1 = PNT1A_OF_ICCM.shape[1]

filtPNT1A_OF_ICCM = np.asmatrix(PNT1A_OF_ICCM)

for i range(1,numrow1):
    
  filtPNT1A_OF_ICCM[i,] == savgol_filter(PNT1A_OF_ICCM[i,],13, 5, 0)


#Baseline Correction 
##PNT1A.OF.ICCM <- baseline(filtPNT1A.OF.ICCM, lambda=2, hwi=210,  it=5,  int=1050,method='fillPeaks')
##PNT1A.OF.ICCM <- PNT1A.OF.ICCM@corrected

#NORMALIZE spectra (vector normalization)
##PNT1A.0Gy.ICCM=sweep(PNT1A.0Gy.ICCM, MARGIN=1, apply(PNT1A.0Gy.ICCM,1,function(x) sqrt(sum(x^2))) , FUN="/") #Vector normalisation function(x)sqrt(sum(x^2))
##PNT1A.OF.ICCM=sweep(PNT1A.OF.ICCM, MARGIN=1, apply(PNT1A.OF.ICCM,1,function(x) sqrt(sum(x^2))) , FUN="/") #Vector normalisation function(x)sqrt(sum(x^2))


This section is similar to the previous sections, simply using new datasets. The problems and solutions are identical to the other datasets

In [ ]:
#FOUR WAY PCA PNT1A in each of 4 classes 
numrow = PNT1A_0Gy.shape[0]
vec1 = {}
vec1[tuple(np.linspace(1,numrow))] = "PNT1A 0Gy"

numrow2 = PNT1A_0Gy_ICCM.shape[0]
vec2 = {}
vec2[tuple(np.linspace(1,numrow2))] = "PNT1A 0Gy + ICCM"

numrow3 = PNT1A_OF_shape[0]
vec3 = {}
vec3[tuple(np.linspace(1,numrow3))] = "PNT1A OF"

numrow4 = PNT1A_OF_ICCM.shape[0]
vec4 = {}
vec4[tuple(np.linspace(1,numrow4))] = "PNT1A OF + ICCM"


dataall = np.concatenate([PNT1A_0Gy,PNT1A_0Gy_ICCM,PNT1A_OF,PNT1A_OF_ICCM])
ID = np.concatenate((vec1, vec2, vec3, vec4), axis=None).astype(str)

#PCA 
##data.pca <- prcomp(dataall ,center = TRUE,scale = FALSE)
##variance=data.pca$sdev*data.pca$sdev
##pvar=variance/sum(variance)
##totvar=sum(pvar[1:5])

Again, similar issues as with the above sections. The plotting methods havent been converted, the focus being on the PCA, Baseline Correction, and Normalization.